In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
path = "./gdrive/MyDrive/iitisoc/"

In [ ]:
import os
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, clean_dir, noisy_dir):
        super(ImageDataset, self).__init__()
        self.clean_file_filenames = [os.path.join(clean_dir, x) for x in os.listdir(clean_dir)]
        self.noisy_file_filenames = [os.path.join(noisy_dir, x) for x in os.listdir(noisy_dir)]
        self.transform = transforms.ToTensor()
    def __getitem__(self, index):
        clean = Image.open(self.clean_file_filenames[index])
        noisy = Image.open(self.noisy_file_filenames[index])
        return self.transform(noisy), self.transform(clean)
    def __len__(self):
        return len(self.clean_file_filenames)

In [ ]:
# -*- coding: utf-8 -*-
"""ADNet Model.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1TwMBYw0Dos-VmalGcf2GAThpdtOzzYx-
"""

import torch
import torch.nn as nn

class Conv_BN_Relu_first(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,padding,groups,bias):
        super(Conv_BN_Relu_first,self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        groups =1 
        self.conv = nn.Conv2d(in_channels=3, out_channels=features, kernel_size=kernel_size, padding=padding,groups=groups, bias=False)
        self.bn = nn.BatchNorm2d(features)
        self.relu = nn.ReLU(inplace=True)
    def forward(self,x):
        return self.relu(self.bn(self.conv(x)))

class Conv_BN_Relu_other(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,padding,groups,bias):
        super(Conv_BN_Relu_other,self).__init__()
        kernel_size = 3
        padding = 1
        features = out_channels
        groups =1 
        self.conv = nn.Conv2d(in_channels=3, out_channels=features, kernel_size=kernel_size, padding=padding,groups=groups, bias=False)
        self.bn = nn.BatchNorm2d(features)
        self.relu = nn.ReLU(inplace=True)
    def forward(self,x):
        return self.relu(self.bn(self.conv(x)))


class Conv(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,padding,groups,bais):
        super(Conv,self).__init__()
        kernel_size = 3
        padding = 1
        features = 1
        groups =1 
        self.conv = nn.Conv2d(in_channels=3, out_channels=features, kernel_size=kernel_size, padding=padding,groups=groups, bias=False)
    def forward(self,x):
        return self.conv(x)

class Self_Attn(nn.Module):
    def __init__(self,in_dim):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        self.query_conv = nn.Conv2d(in_channels=in_dim,out_channels=in_dim//8,kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim,out_channels=in_dim//8,kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim,out_channels=in_dim,kernel_size=1)
        self.gamma=nn.Parameter(torch.zeros(1))
        self.softmax=nn.Softmax(dim=-1)
    def forward(self,x):
        m_batchsize, C, width,height = x.size()
        proj_query = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1)
        proj_key = self.key_conv(x).view(m_batchsize,-1,width*height)
        
        energy = torch.bmm(proj_query,proj_key)
        attention = self.softmax(energy)
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) 
        out = torch.bmm(proj_value,attention.permute(0,2,1))
        out = out.view(m_batchsize,C,width,height)
        out = self.gamma*out + x
        return out, attention

class ADNet(nn.Module):
  def __init__(self, channels, num_of_layers=15):
        super(ADNet, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        groups =1 
        layers = []
        kernel_size1 = 1
        self.conv1_1 = nn.Sequential(nn.Conv2d(in_channels=channels,out_channels=features,kernel_size=kernel_size,padding=padding,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_2 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=2,groups=groups,bias=False,dilation=2),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_3 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=1,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_4 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=1,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_5 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=2,groups=groups,bias=False,dilation=2),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_4 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=1,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_5 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=2,groups=groups,bias=False,dilation=2),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_6 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=1,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_7 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=padding,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_8 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=1,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_9 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=2,groups=groups,bias=False,dilation=2),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_10 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=1,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_11 = nn.Sequential(nn.Conv2d(in_channels=features,out_channels=features,kernel_size=kernel_size,padding=1,groups=groups,bias=False),nn.BatchNorm2d(features),nn.ReLU(inplace=True))
        self.conv1_12 = nn.Conv2d(in_channels=features,out_channels=3,kernel_size=kernel_size,padding=1,groups=groups,bias=False)
        self.conv3 = nn.Conv2d(in_channels=6,out_channels=3,kernel_size=1,stride=1,padding=0,groups=1,bias=True)
        self.ReLU = nn.ReLU(inplace=True)
        self.Tanh= nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                
                m.weight.data.normal_(0, (2 / (9.0 * 64)) ** 0.5)
            if isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(0, (2 / (9.0 * 64)) ** 0.5)
                clip_b = 0.025
                w = m.weight.data.shape[0]
                for j in range(w):
                    if m.weight.data[j] >= 0 and m.weight.data[j] < clip_b:
                        m.weight.data[j] = clip_b
                    elif m.weight.data[j] > -clip_b and m.weight.data[j] < 0:
                        m.weight.data[j] = -clip_b
                m.running_var.fill_(0.01)

  def _make_layers(self, block,features, kernel_size, num_of_layers, padding=1, groups=1, bias=False):
      layers = []
      for _ in range(num_of_layers):
        layers.append(block(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, groups=groups, bias=bias))
      return nn.Sequential(*layers)                
    
      
	  
  def forward(self, x):
        input = x 
        x1 = self.conv1_1(x)
        x1 = self.conv1_2(x1)
        x1 = self.conv1_3(x1)
        x1 = self.conv1_4(x1)
        x1 = self.conv1_5(x1)
        x1 = self.conv1_6(x1)
        x1 = self.conv1_7(x1)   
        x1 = self.conv1_8(x1)
        x1 = self.conv1_9(x1)
        x1 = self.conv1_10(x1)
        x1 = self.conv1_11(x1)
        x1 = self.conv1_12(x1)
        
        out = torch.cat([x,x1],1)
        out= self.Tanh(out)
        out = self.conv3(out)
        out = out*x1
        out2 = x - out
        return out2

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import transforms

from math import log10
import matplotlib.pyplot as plt


train_dataset = ImageDataset(path + "/large/clean/", path + "/large/noisy/")
test_dataset = ImageDataset(path + "/large/test_clean/", path + "/large/test_noisy/")

batch_size = 1

training_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
testing_data_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ADNet(channels=3, num_of_layers=4).to(device)

criterion = nn.MSELoss()
lr = 1e-4
optimizer = optim.Adam(model.parameters(), lr=lr)


def train(epoch):
    epoch_loss = 0
    for i, data in enumerate(training_data_loader, 1):
        input = data[0].cuda()
        target = data[1].cuda()
        out = model(input)

        loss = criterion(out, target)
        epoch_loss += loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, i, len(training_data_loader), loss.item()))
    
    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))

def validate():
    avg_psnr = 0
    model.eval()
    with torch.no_grad():
        for data in testing_data_loader:
            input = data[0].cuda()
            output = model(input)
            mse = criterion(output, input)
            psnr = 10 * log10(1 / mse.item())
            avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))
    model.train()


def save(state, epoch):
    model_out_path = "./model_epoch_{}.pth".format(epoch)
    torch.save(state, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

def training():
    checkpoint = torch.load("./model_epoch_60.pth")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']

    num_epochs = 20
    for epoch in range(61, num_epochs + 61):
        train(epoch)
        if epoch % 10 == 0:
            validate()
            save({
                'epoch': epoch + 1,
                'arch': model,
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict(),
            }, epoch)

def testing():
    for image_number in range(320,330):
        trans = transforms.ToPILImage()
        model = torch.load("./model_epoch_20.pth")
        model = model['arch']
        loader = transforms.Compose([
            transforms.ToTensor()
        ])
        image = Image.open(f"../new_data/final_test/noisy_{image_number}.jpg")
        image = loader(image).float()
        print(image.shape)
        image = image.unsqueeze(0)
        out = torch.clamp(image-model(image), 0., 1.)
        out = out.squeeze(0)
        print(out.shape)
        trans(out).save(f'../new_data/final_test/clean_{image_number}.jpg')

In [ ]:
training()

===> Epoch[61](1/32): Loss: 0.0204
===> Epoch[61](2/32): Loss: 0.0245
===> Epoch[61](3/32): Loss: 0.0189
===> Epoch[61](4/32): Loss: 0.0390
===> Epoch[61](5/32): Loss: 0.0232
===> Epoch[61](6/32): Loss: 0.0359
===> Epoch[61](7/32): Loss: 0.0339
===> Epoch[61](8/32): Loss: 0.0404
===> Epoch[61](9/32): Loss: 0.0393
===> Epoch[61](10/32): Loss: 0.0771
===> Epoch[61](11/32): Loss: 0.0439
===> Epoch[61](12/32): Loss: 0.0404
===> Epoch[61](13/32): Loss: 0.0370
===> Epoch[61](14/32): Loss: 0.0493
===> Epoch[61](15/32): Loss: 0.0165
===> Epoch[61](16/32): Loss: 0.0156
===> Epoch[61](17/32): Loss: 0.0150
===> Epoch[61](18/32): Loss: 0.0253
===> Epoch[61](19/32): Loss: 0.0238
===> Epoch[61](20/32): Loss: 0.0557
===> Epoch[61](21/32): Loss: 0.0434
===> Epoch[61](22/32): Loss: 0.0466
===> Epoch[61](23/32): Loss: 0.0435
===> Epoch[61](24/32): Loss: 0.0311
===> Epoch[61](25/32): Loss: 0.0473
===> Epoch[61](26/32): Loss: 0.0370
===> Epoch[61](27/32): Loss: 0.0334
===> Epoch[61](28/32): Loss: 0.0352
=

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')